Business Problem:
Analyse order delivery performance and customer satisfaction to identify delivery delays,revenue trends,and key improvement areas.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt



STEP 1: IMPORTING CSV FILES AND CHECKING SHAPES & FIRST 5 ROWS

In [4]:
import pandas as pd

# Load CSV files
orders = pd.read_csv(r"D:\Data_Analysis_Projects\E-Commerce analysis project\01_Data\olist_orders_dataset.csv")
order_items = pd.read_csv(r"D:\Data_Analysis_Projects\E-Commerce analysis project\01_Data\olist_order_items_dataset.csv")
customers = pd.read_csv(r"D:\Data_Analysis_Projects\E-Commerce analysis project\01_Data\olist_customers_dataset.csv")
reviews = pd.read_csv(r"D:\Data_Analysis_Projects\E-Commerce analysis project\01_Data\olist_order_reviews_dataset.csv")

# Print shapes
print("Orders shape:", orders.shape)
print("Order Items shape:", order_items.shape)
print("Customers shape:", customers.shape)
print("Reviews shape:", reviews.shape)

# Display first 5 rows
orders.head()
order_items.head()
customers.head()
reviews.head()

Orders shape: (99441, 8)
Order Items shape: (112650, 7)
Customers shape: (99441, 5)
Reviews shape: (99224, 7)


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


STEP 2: DATA QUALITY CHECK(NULL VALUES & DATATYPES)

In [5]:
# Function to create summary table
def data_quality_summary(df):
    summary = pd.DataFrame({
        "Column Name": df.columns,
        "Null Count": df.isnull().sum().values,
        "Data Type": df.dtypes.values
    })
    return summary

# Create summaries
orders_quality = data_quality_summary(orders)
order_items_quality = data_quality_summary(order_items)
customers_quality = data_quality_summary(customers)
reviews_quality = data_quality_summary(reviews)

orders_quality.head()


,Column Name,Null Count,Data Type
0,order_id,0,object
1,customer_id,0,object
2,order_status,0,object
3,order_purchase_timestamp,0,object
4,order_approved_at,160,object


** order_approved_at has null values because some orders were not approved or were cancelled.
** These are valid business scenarios, not data errors.
** Since this column is not used in delivery calculations, null values are intentionally left as-is.



In [6]:
# DUPLICATE CHECK
print("Duplicate orders:", orders.duplicated().sum())
print("Duplicate order items:", order_items.duplicated().sum())
print("Duplicate customers:", customers.duplicated().sum())
print("Duplicate reviews:", reviews.duplicated().sum())


Duplicate orders: 0
Duplicate order items: 0
Duplicate customers: 0
Duplicate reviews: 0


In [7]:
# DATE COLUMN INSPECTION
orders[['order_purchase_timestamp',
        'order_delivered_customer_date',
        'order_estimated_delivery_date']].head()


,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date
0,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00
1,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00
2,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00
3,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00
4,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00


STEP 3: DATA CLEANING

In [8]:
# orders table
# Convert date columns to datetime
date_cols = [
    'order_purchase_timestamp',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col])

# Create delivery_days
orders['delivery_days'] = (
    orders['order_delivered_customer_date'] -
    orders['order_purchase_timestamp']
).dt.days

# Create delay_flag
orders['delay_flag'] = orders['order_delivered_customer_date'] > orders['order_estimated_delivery_date']
orders['delay_flag'] = orders['delay_flag'].map({True: "Yes", False: "No"})


In [9]:
# order_items table
# Create revenue column
order_items['revenue'] = order_items['price'] + order_items['freight_value']
# Display order_items revenue calculation
order_items[['order_id', 'price', 'freight_value', 'revenue']].head()


,order_id,price,freight_value,revenue
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14,218.04


In [10]:
# reviews table
# Keep review_score, fill missing comments with blank
reviews['review_comment_message'] = reviews['review_comment_message'].fillna("")
# Display review score and cleaned comment column
reviews[['order_id', 'review_score', 'review_comment_message']].head()



,order_id,review_score,review_comment_message
0,73fc7af87114b39712e6da79b0a377eb,4,
1,a548910a1c6147796b98fdf73dbeba33,5,
2,f9e4b658b201a9f2ecdecbb34bed034b,5,
3,658677c97b385a9be170737859d3511b,5,Recebi bem antes do prazo estipulado.
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,Parabéns lojas lannister adorei comprar pela I...


In [11]:
# customers table
# Standardize city and state text
customers['customer_city'] = customers['customer_city'].str.lower()
customers['customer_state'] = customers['customer_state'].str.lower()
customers.head()


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,sp
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,sp
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,sp
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,sp
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,sp


STEP 4: MERGING TABLES BASED ON COMMON CLOUMN

In [12]:
# Orders is the base table
# Joining other tables using order_id

df = orders.merge(order_items, on='order_id', how='left')
df = df.merge(customers, on='customer_id', how='left')
df = df.merge(reviews, on='order_id', how='left')

df.shape
df.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_days,delay_flag,...,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.0,No,...,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,sp,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.0,No,...,af07308b275d755c9edb36a90c618231,47813,barreiras,ba,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9.0,No,...,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,go,e73b67b67587f7644d5bd1a52deb1b01,5.0,NaN,,2018-08-18 00:00:00,2018-08-22 19:07:58
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13.0,No,...,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,rn,359d03e676b3c069f62cadba8dd3f6e8,5.0,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2.0,No,...,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,sp,e50934924e227544ba8246aeb3770dd4,5.0,NaN,,2018-02-17 00:00:00,2018-02-18 13:02:51


Orders table is the base; other tables are joined using order_id.

STEP 5: FUTURE ENGENEERING

In [13]:
# Extract Order year and month
df['order_year'] = df['order_purchase_timestamp'].dt.year
df['order_month'] = df['order_purchase_timestamp'].dt.month

# Delivery status
df['delivery_status'] = df['delay_flag'].map({
    "Yes": "Delayed",
    "No": "On-time"
})

# Review category
def review_category(score):
    if score <= 2:
        return "Poor"
    elif score == 3:
        return "Neutral"
    else:
        return "Good"

df['review_category'] = df['review_score'].apply(review_category)

# Display data with new columns
df.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_days,delay_flag,...,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,order_year,order_month,delivery_status,review_category
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.0,No,...,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,2017,10,On-time,Good
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.0,No,...,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,2018,7,On-time,Good
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9.0,No,...,e73b67b67587f7644d5bd1a52deb1b01,5.0,NaN,,2018-08-18 00:00:00,2018-08-22 19:07:58,2018,8,On-time,Good
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13.0,No,...,359d03e676b3c069f62cadba8dd3f6e8,5.0,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58,2017,11,On-time,Good
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2.0,No,...,e50934924e227544ba8246aeb3770dd4,5.0,NaN,,2018-02-17 00:00:00,2018-02-18 13:02:51,2018,2,On-time,Good


In this step, simple analytical features were created from existing columns to support time-based analysis, delivery performance comparison, and customer satisfaction grouping.
These features improve dashboard clarity without adding complexity.

In [14]:
df.to_csv("olist_cleaned_for_powerbi.csv", index=False)


This dataset is used for Power BI analysis